$$S_i = S_{0i} e^{-(\alpha_i + j\omega_i)t}e^{j\phi_i}$$

$$S_i = S_{0i} e^{-(\alpha_i + j\omega_i)kT_s}e^{j\phi_i}$$

$$S_i = S_{0i} e^{(-(\alpha_i + j\omega_i)T_s)k}e^{j\phi_i}$$

$$e^{jx} = cos(x) + jsin(x)$$

$$e^{y + jx} = e^ye^{jx} = e^y(cos(x) + jsin(x))$$

$$e^{-\alpha_i - j\omega_i} = e^{-\alpha}(cos(\omega) + jsin(\omega))$$

Considerando o cálculo de um polo com valor $z = a + jb$:

$$e^{-\alpha}(cos(\omega) + jsin(\omega)) = a + jb$$

$$e^{-\alpha}cos(\omega) = a$$

$$e^{-\alpha}sin(\omega) = b$$

$$-\alpha + \log(cos(\omega)) = \log(a)$$

$$\alpha = \log(cos(\omega)) - \log(a)$$

$$e^{-\alpha}sin(\omega) = b$$

$$\alpha = \log(sin(\omega)) - log(b)$$

No final, obtemos:

$$\alpha = \log(cos(\omega)) - \log(a)$$

$$\alpha = \log(sin(\omega)) - \log(b)$$

Subtraindo a primeira da segunda:

$$0 = \log(sin(\omega)) - \log(b) - \log(cos(\omega)) + \log(a)$$

$$0 = \log(\frac{sin(\omega)}{cos(\omega)}) - \log(b) + \log(a)$$

$$0 = \log(tan(\omega)) + \log(\frac{a}{b})$$

$$1 = \frac{a \ tan(\omega)}{b}$$

$$\omega = \arctan(\frac{b}{a})$$

Que nos permite também solucionar para $\alpha$:

$$\alpha = \log(\frac{cos(\arctan(\frac{b}{a}))}{a})$$



Considerando agora o cálculo de um resíduo $R = c + jd$:

$$S_0 (cos(\phi) + jsin(\phi)) = c + jd$$

$$S_0 cos(\phi) = c$$

$$S_0 sin(\phi) = d$$

Fazendo a razão da segunda pela primeira:

$$\frac{S_0}{S_0}\frac{sin(\phi)}{cos(\phi)} = \frac{d}{c}$$

$$\phi = \arctan(\frac{d}{c})$$

Substituindo na primeira equação:

$$S_0 = \frac{c}{cos(\arctan(\frac{d}{c}))}$$

Sendo assim, considerando o cálculo de polos e resíduos da forma:

$$z = a + jb$$

$$R = c + jd$$

Temos:

| $S_0$ | $\phi$ | $\omega$ | $\alpha$ | 
| ----- | ------ | -------- | -------- |
| $\frac{c}{cos(\arctan(\frac{d}{c}))}$ | $\arctan(\frac{d}{c})$ | $\arctan(\frac{b}{a})$ | $\log(\frac{cos(\arctan(\frac{b}{a}))}{a})$ | 
